## Create a Mongo Database from a CSV

In [1]:
import os
import pandas as pd
from pymongo import MongoClient
import json
import csv

In [2]:
def import_content(filepath, mongodb_name, mongodb_collection_name):
    mng_client = MongoClient('localhost', 27017)
    mng_db = mng_client[f'{mongodb_name}']  # Replace mongo db name
    collection_name = mongodb_collection_name  # Replace mongo db collection name
    db_cm = mng_db[collection_name]

    data = pd.read_csv(filepath, na_filter=False)
    data_json = json.loads(data.to_json(orient='records'))
    db_cm.remove()
    db_cm.insert(data_json)
    

Take the cleaned corpus from Summit Post and put it into a Mongo database collection called cleaned_corpus.

In [3]:
import_content("data/cleaned_corpus.csv", mongodb_name = "summitdb", mongodb_collection_name= "cleaned_corpus")

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if __name__ == '__main__':
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  # Remove the CWD from sys.path while we load stuff.


Check and see what the first few entries look like

In [4]:
client = MongoClient()

In [5]:
db = client.summitdb
db.collection_names()

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  


['cleaned_corpus']

In [ ]:
'''run the following in the mongo shell to remove the 'Unnamed' field'''
# db.cleaned_corpus.update({}, {$unset: {'Unnamed: 0':1}} , {multi: true});

In [7]:
cursor = db.cleaned_corpus.find({}, {'_id':0}).limit(30)

list(cursor)[:5]

[{'mountain': 'Mount Whitney',
  'comment': 'did this in a single day very difficult for a single day hike especially for me cause this was my very first time at such high altitudes diamox helped to prevent altitude sicknessspotted a bear at trailhead too which was a bit scary ',
  'comment_date': '2018-11-19 08:51:00',
  'climb_date': '2015-09-05 00:00:00'},
 {'mountain': 'Mount Whitney',
  'comment': 'mountaineers route  first ',
  'comment_date': '2018-11-01 06:34:00',
  'climb_date': ''},
 {'mountain': 'Mount Whitney',
  'comment': 'standard route from whitney portal via trail camp made it a day trip staying the night at trail camp before summit daypretty straightforward in ideal conditions this is not a difficult summit at all if you are welltrained no water available above trail camp especially this time of yearfrom trail camp it was just under  hours for us to reach the jmt from there another  minutes to the summit quite leisurely as this is by far the highest my wife has ever b

In [8]:
# to aggregate documents by mountain

cursor = db.cleaned_corpus.aggregate([
     { "$group" : { "_id" : "$mountain", "cleaned_corpus": { "$push": "$$ROOT" } } }])

list(cursor)[:5]

[{'_id': 'Green Mountain',
  'cleaned_corpus': [{'_id': ObjectId('5c75ecd257ce423b68107526'),
    'mountain': 'Green Mountain',
    'comment': 'just an afternoon of trail running  hiking ',
    'comment_date': '2017-11-14 01:00:00',
    'climb_date': '2017-10-24 00:00:00'},
   {'_id': ObjectId('5c75ecd257ce423b68107527'),
    'mountain': 'Green Mountain',
    'comment': 'went up the short west ridge but came back down the em greenman for a big loop fun hike with great views',
    'comment_date': '2017-07-08 19:14:00',
    'climb_date': '2017-06-26 00:00:00'},
   {'_id': ObjectId('5c75ecd257ce423b68107528'),
    'mountain': 'Green Mountain',
    'comment': 'my wife is applying to grad school at cuboulder so we came out to check it out boulder was great and the mountains around the town are fantastic chose to hike green because it seems like a local staple the west ridge trail was easy and the views from the top are great i loved gazing at the continental divide longs peak looms large ev